In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
import os
os.chdir('/gdrive/My Drive/Sentihood')

In [3]:
import pandas as pd
import json
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
MODEL_DIR = 'run_roberta_multi_20'
df_test = pd.read_csv(os.path.join(MODEL_DIR, 'predictions.csv'))
df_test.head(5)

,id,text,target,aspect,sentiment,pred_aspect,pred_sentiment
0,153,LOCATION1 is in Greater London and is a very safe place,LOCATION1,safety,Positive,safety,Positive
1,1130,All the neighborhoods around LOCATION1 are very nice and safe,LOCATION1,general safety,Positive Positive,general safety,Positive Positive
2,1271,"Cheap is LOCATION2, LOCATION1, but not really cool",LOCATION1,general price,Negative Positive,general,Negative
3,1271,"Cheap is LOCATION2, LOCATION1, but not really cool",LOCATION2,price general,Positive Negative,general,Negative
4,1089,Dont Try LOCATION1,LOCATION1,general,Negative,general,Negative


In [5]:
df_test[df_test['id']==1271]

,id,text,target,aspect,sentiment,pred_aspect,pred_sentiment
2,1271,"Cheap is LOCATION2, LOCATION1, but not really cool",LOCATION1,general price,Negative Positive,general,Negative
3,1271,"Cheap is LOCATION2, LOCATION1, but not really cool",LOCATION2,price general,Positive Negative,general,Negative


In [6]:
with open('test.json') as f:
  data = json.load(f)

In [7]:
df_test['pred_aspect'].fillna('None', inplace=True)
df_test['pred_sentiment'].fillna('None', inplace=True)

In [8]:
json_list = []
c = 0

for d in data:
  idx = d['id']
  df1 = df_test[df_test['id']==idx].reset_index()
  opinions = d['opinions']
  text = d['text']

  model_preds = []

  for _, row in df1.iterrows():
    pred_aspect = str(row['pred_aspect']).split(' ')
    pred_sentiment = str(row['pred_sentiment']).split(' ')

    t = {}
    for i, j in zip(pred_aspect, pred_sentiment):
      if i=='None' or j=='None' or i=='' or j=='': # use prediction only when both aspect and sentiment are available
        continue
      t['target_entity'] = row['target']
      t['aspect'] = i
      t['sentiment'] = j
      model_preds.append(t)

  if len(model_preds)==0:
    c+=1

  ans = {'opinions': opinions,
        'id': idx,
        'text': text,
        'model_pred':model_preds,
        }
  json_list.append(ans)


In [9]:
print(f'Model did not predict any aspect for {c} instances')
df_test[df_test['pred_aspect']=='None'].sample(5)

Model did not predict any aspect for 189 instances


,id,text,target,aspect,sentiment,pred_aspect,pred_sentiment
1628,350,London Eye to one side LOCATION1 to the other,LOCATION1,price,Positive,None,None
1803,1117,because it was filmed andd based in LOCATION1 in London,LOCATION1,price,Positive,None,None
1672,610,Plus unlike LOCATION2 which is all houses theres tons of flats in LOCATION1,LOCATION1,price,Positive,None,None
752,50,"LOCATION1 is not a `` London '' in itself , it is simply a town within the north-east surburbs of London",LOCATION1,transit-location,Negative,None,None
1422,957,"I know LOCATION1 Broadway quite well, but not the other two places",LOCATION1,price,Positive,None,None


In [10]:
with open(os.path.join(MODEL_DIR, 'preds.jsonl'), 'w') as f:
  for js in json_list:
    json_record = json.dumps(js, ensure_ascii=False) 
    f.write(json_record + '\n')

In [11]:
! head {os.path.join(MODEL_DIR, 'preds.jsonl')}

{"opinions": [{"sentiment": "Positive", "aspect": "safety", "target_entity": "LOCATION1"}], "id": 153, "text": "   LOCATION1 is in Greater London  and is a very safe place", "model_pred": [{"target_entity": "LOCATION1", "aspect": "safety", "sentiment": "Positive"}]}
{"opinions": [{"sentiment": "Positive", "aspect": "general", "target_entity": "LOCATION1"}, {"sentiment": "Positive", "aspect": "safety", "target_entity": "LOCATION1"}], "id": 1130, "text": "  All the neighborhoods around LOCATION1 are very nice and safe", "model_pred": [{"target_entity": "LOCATION1", "aspect": "safety", "sentiment": "Positive"}, {"target_entity": "LOCATION1", "aspect": "safety", "sentiment": "Positive"}]}
{"opinions": [{"sentiment": "Negative", "aspect": "general", "target_entity": "LOCATION1"}, {"sentiment": "Positive", "aspect": "price", "target_entity": "LOCATION1"}, {"sentiment": "Positive", "aspect": "price", "target_entity": "LOCATION2"}, {"sentiment": "Negative", "aspect": "general", "target_entity"